In [14]:
import numpy as np 
import pandas as pd
import os
import torch
import numpy
pd.set_option('display.max_rows', None)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [15]:
df_train=pd.read_csv("/kaggle/input/titanic/train.csv")
df_test=pd.read_csv("/kaggle/input/titanic/test.csv")
df_train.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [16]:
# print(df_train['Pclass'].unique())
# print(df_train['SibSp'].unique())
# print(df_train['Parch'].unique())
# print(df_train['Cabin'].unique())

df_train=df_train.drop('Ticket',axis=1)
df_train=df_train.drop('PassengerId',axis=1)
df_train=df_train.drop('Name',axis=1)
df_train.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,7.2500,NaN,S
1,1,1,female,38.0,1,0,71.2833,C85,C
2,1,3,female,26.0,0,0,7.9250,NaN,S
3,1,1,female,35.0,1,0,53.1000,C123,S
4,0,3,male,35.0,0,0,8.0500,NaN,S


Cabin: Nan and 大写字母(A-G)+数字
Embarked: train:"Q","S","C", test:Nan,"Q","S","C", ticket字段要删掉, 
Age: Nan and others

In [17]:
df_test=df_test.drop('Ticket',axis=1)
#df_test=df_test.drop('PassengerId',axis=1)
df_test=df_test.drop('Name',axis=1)
df_test.head(5)

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,892,3,male,34.5,0,0,7.8292,NaN,Q
1,893,3,female,47.0,1,0,7.0000,NaN,S
2,894,2,male,62.0,0,0,9.6875,NaN,Q
3,895,3,male,27.0,0,0,8.6625,NaN,S
4,896,3,female,22.0,1,1,12.2875,NaN,S


In [18]:
df_train['Sex']=df_train['Sex'].replace({'female':0, 'male':1})
df_train['Embarked']=df_train['Embarked'].replace({'C':1,'S':2,'Q':3})
df_train['Embarked']=df_train['Embarked'].fillna(0.0)
df_train['Age']=df_train['Age'].fillna(0.0)
df_train['Pclass']=df_train['Pclass'].fillna(0.0)

df_test['Sex']=df_test['Sex'].replace({'female':0, 'male':1})
df_test['Embarked']=df_test['Embarked'].replace({'C':1,'S':2,'Q':3})
df_test['Embarked']=df_test['Embarked'].fillna(0.0)
df_test['Age']=df_test['Age'].fillna(0.0)
df_test['Pclass']=df_test['Pclass'].fillna(0.0)

def encode_category(category):
    return ord(category[0]) - ord('A')  # ord()是获取该字母的unicode编码值

df_train['Cabin'] = df_train['Cabin'].fillna('A') #空值替换成字母G
df_test['Cabin'] = df_test['Cabin'].fillna('A')
df_train['Cabin'] = df_train['Cabin'].map(encode_category) #将该字段的首字母改成数字
df_test['Cabin'] = df_test['Cabin'].map(encode_category)
#df_train['Age']
#df_train.head(5)


In [19]:
train_data=df_train.values
test_data=df_test.values
print(train_data[:5])
print(test_data[:5])

[[ 0.      3.      1.     22.      1.      0.      7.25    0.      2.    ]
 [ 1.      1.      0.     38.      1.      0.     71.2833  2.      1.    ]
 [ 1.      3.      0.     26.      0.      0.      7.925   0.      2.    ]
 [ 1.      1.      0.     35.      1.      0.     53.1     2.      2.    ]
 [ 0.      3.      1.     35.      0.      0.      8.05    0.      2.    ]]
[[892.       3.       1.      34.5      0.       0.       7.8292   0.
    3.    ]
 [893.       3.       0.      47.       1.       0.       7.       0.
    2.    ]
 [894.       2.       1.      62.       0.       0.       9.6875   0.
    3.    ]
 [895.       3.       1.      27.       0.       0.       8.6625   0.
    2.    ]
 [896.       3.       0.      22.       1.       1.      12.2875   0.
    2.    ]]


In [20]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import numpy as np
class TrainDataset(Dataset):
    def __init__(self,train_data):
        self.train_value=train_data[:,1:]
        self.train_label=train_data[:,0].astype(int)
    def __len__(self):
        return self.train_value.shape[0]
    def __getitem__(self,index):
        data=self.train_value[index]
        label=self.train_label[index]
        return {'data':data,'label':label}
    
class TestDataset(Dataset):
    def __init__(self,test_data):
        self.test_value=test_data[:,1:]
        self.test_id=test_data[:,0].astype(int)
    def __len__(self):
        return self.test_value.shape[0]
    def __getitem__(self,index):
        data=self.test_value[index]
        test_id=self.test_id[index]
        return {'data':data,'test_id':test_id} 

def collate_fn(batch):
    data = [item['data'] for item in batch]
    test_id = [item['test_id'] for item in batch]
    return {'data': torch.tensor(data), 'test_id': torch.tensor(test_id)}
train_dataset=TrainDataset(train_data)
test_dataset=TestDataset(test_data)
train_loader=DataLoader(train_dataset,batch_size=64,shuffle=True,num_workers=2,drop_last=True,pin_memory=True)
test_loader=DataLoader(test_dataset,batch_size=test_data.shape[0],num_workers=2,collate_fn=collate_fn,pin_memory=True)

In [21]:
import torch.nn.functional as F
import torch
import torch.optim.lr_scheduler as lr_sheduler
import torch.nn as nn
from torch.autograd import Variable
from tqdm import tqdm
class TitanicNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.norm = nn.LayerNorm(8)
        self.mlp_module = nn.Sequential(
                             nn.Linear(8,128),
                             nn.Dropout(0.5),
                             nn.LeakyReLU(),
                             nn.Linear(128,64),
                             nn.Dropout(0.5),
                             nn.LeakyReLU(),
                             nn.Linear(64,64),
                             nn.Dropout(0.5),
                             nn.LeakyReLU(),
                             nn.Linear(64,64),
                             nn.Dropout(0.5),
                             nn.LeakyReLU(),
                             nn.Linear(64,8),
                             nn.Dropout(0.5),
                             nn.LeakyReLU()) 
        self.mlp_predict = nn.Sequential(nn.Linear(8,2)) # 最后的预测结果不需要激活函数和BatchNorm
        
    def forward(self, batch):
        x=batch['data'].type(torch.float32).to(device) # (B,8)
        x=self.norm(x)
        x=self.mlp_module(x)
        x=self.mlp_predict(x)
        return x
    
model = TitanicNet()
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001,betas=(0.9, 0.999))
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20,gamma=0.3) #学习率调度器，在指定的步长(step_size)处乘以衰减系数(gamma)
loss=0
for epoch in range(0, 200):
    with tqdm(enumerate(train_loader), total=len(train_loader), ncols=150, position=0, leave=True) as t:     
        for i,data in t:
            torch.cuda.synchronize()
            data = {key:Variable(value, requires_grad=False) for key, value in data.items()}
            predict = model(data)
            loss = F.cross_entropy(predict,data['label'].to(device))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            torch.cuda.synchronize()
    scheduler.step()
print("loss:",loss)
model.eval()
with torch.no_grad():
    for _, batch in enumerate(test_loader):
        result = model(batch)
        result = result.argmax(dim=1)
        output=torch.vstack(((batch["test_id"]).to(device),result)).t()
        header="PassengerId,Survived"
        output=output.cpu().numpy()
        print(output.shape)
        np.savetxt("submission.csv",output, delimiter=",", header=header, comments="",fmt="%d")
        print("Your submission was successfully saved!")


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 124.53it/s]

loss: tensor(0.6040, device='cuda:0', grad_fn=<NllLossBackward0>)



/tmp/ipykernel_28/766179106.py:29: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:245.)
  return {'data': torch.tensor(data), 'test_id': torch.tensor(test_id)}


(418, 2)
Your submission was successfully saved!
